In [108]:
import sys
import os
import time
import pandas as pd
sys.path.insert(0, os.path.realpath('..'))
import django
django.setup()
from django.db import connections
from pretty import pprint
from firecares.firestation.models import FireDepartment, FireStation, NFIRSStatistic
from django.db.models import Avg, Max, Min, Q
from IPython.display import display
from firecares.utils import lenient_summation
pd.set_option("display.max_rows",100)

#### Predictions 2015 csv processing

In [125]:
fd = FireDepartment.objects.get(id=93345)
df = pd.read_csv('/firecares/predictions.2015.csv')
cols = ['lr_fire', 'mr_fire', 'h.fire', 'lr_inj', 'mr_inj', 'h.inj', 'lr_death', 'mr_death', 'h.death', 'lr_size_2', 'mr_size_2', 'h.size2', 'lr_size_3', 'mr_size_3', 'h.size3']
# Find stats for Richmond, VA
richmond = df[df['fd_id'] == 93345]

In [124]:
# Sum all Richmond rows
df2 = richmond.groupby(['fd_id', 'state'])[cols].sum()
df2

,,lr_fire,mr_fire,h.fire,lr_inj,mr_inj,h.inj,lr_death,mr_death,h.death,lr_size_2,mr_size_2,h.size2,lr_size_3,mr_size_3,h.size3
fd_id,state,,,,,,,,,,,,,,,
93345,VA,166.284421,NaN,177.324624,45.619757,4.770429,0.905018,1.747018,NaN,1.279571,29.837817,NaN,110.997977,7.132377,NaN,107.042456


In [123]:
# High standard deviation for high-risk-level fire values
richmond.std()

Unnamed: 0    19.411049
fd_id          0.000000
lr_fire        1.786107
mr_fire             NaN
lr_inj         0.467798
mr_inj         0.043219
lr_death       0.016622
mr_death            NaN
lr_size_2      0.025631
lr_size_3      0.013128
mr_size_2           NaN
mr_size_3           NaN
h.fire        14.085817
h.inj          0.046699
h.death        0.130074
h.size2        6.108413
h.size3        6.650259
dtype: float64

In [110]:
# Actuals from NFIRS, average of residential structure fires over years (for high structure risk level)

pprint(list(fd.nfirsstatistic_set.filter(metric='residential_structure_fires', year__gte=2010, level=4).values('count', 'year')))

high_avg = fd.nfirsstatistic_set.filter(metric='residential_structure_fires', year__gte=2010, level=4).aggregate(Avg('count')).get('count__avg')
print 'Actual average over high-risk structure types per year: {}'.format(high_avg)

[{'count': 23, 'year': 2014},
 {'count': None, 'year': 2013},
 {'count': 15, 'year': 2012},
 {'count': 22, 'year': 2011},
 {'count': 20, 'year': 2010}]
Actual average over high-risk structure types per year: 20.0


In [121]:
# The current predicted # of fires for high risk structures
print 'Predicted # fires for high-risk structures: {}'.format(sum([df2['h.fire'][0], df2['h.size2'][0], df2['h.size3'][0]]))

Predicted # fires for high-risk structures: 395.365056951


#### Displayed value verification on FireCARES

In [104]:
# Verify "Number of fires -> Average since 2010" displayed values

low = fd.nfirsstatistic_set.filter(metric='residential_structure_fires', year__gte=2010, level=1).aggregate(Avg('count')).get('count__avg')
metrics = fd.metrics.residential_fires_3_year_avg

assert low == metrics.low

display(low)
display(metrics)

85.0

{'all': 251.0, 'high': 20.0, 'low': 85.0, 'medium': 62.0, 'unknown': 117.4}

In [30]:
# Verify predicted deaths and injuries for "Low" structure hazard levels displayed values

low = df2['lr_death'][0] + df2['lr_inj'][0]
assert abs(low - fd.metrics.deaths_and_injuries_sum.low) < 0.0001
display(fd.metrics.deaths_and_injuries_sum)

{'all': 54.3217925804363,
 'high': 2.184589854142297,
 'low': 47.36677412664759,
 'medium': 4.77042859964636,
 'unknown': None}

In [40]:
# Verify sum of death and injuries over all risk levels
v = sum(filter(lambda x: x >= 0, [df2['lr_death'][0], df2['lr_inj'][0], df2['mr_death'][0], df2['mr_inj'][0], df2['h.death'][0], df2['h.inj'][0]]))
assert abs(v - fd.metrics.deaths_and_injuries_sum.all) < 0.0001